# 자연어 처리 NLP (Natural Language Processing)

자연어 처리는 인간의 언어 현상을 컴퓨터와 같은 기계를 이용하여 모사할 수 있도록 연구하고 이를 구현하는 인공지능의 주요 분야 중 하나다.<br>
컴퓨터 공학과 언어학과 같은 다른 학문과의 융합적인 요소도 필요하며, 컴퓨터와 인간 사이의 연결고리를 만들어주는 것이 자연어 처리 분야라고 할 수 있다.

## 자연어 처리의 주 응용분야

* Sentiment Analysis - 감성분석, 인터넷 이용자의 글에서 감정을 분류하는 작업
* Quention & Answering - QA, 지문을 읽고 주어진 질문에 대해 대답하는 분야
* 애플의 Siri와 같이 사용자의 의도를 파악하고 대화하거나 도움을 주는 작업
* Summarization 요약, Machine Translation 기계번역과 같은 작업
* Language Modeling - 언어모델링, 첫 몇 마디만 입력하면 뒤에 나올 단어 예측해서 선택만 하면 되는 스마트보드

1950년부터 자연어를 처리하려는 시도가 있었고 그 뒤로 계속 문제점을 가지고 있다가 <br>
딥러닝이 발전하면서 2014년에 seq2seq(Sequence-to-Sequence)구조가 제시되면서 기계 번역은 신경망을 기반으로 하는 것이 대세가 됨

딥러닝이 처음에 이미지 처리 분야에서 강한 면모를 보였다면 자연어 처리는 최근에 가장 진척이 많이 나간 연구 분야이기도 함<br>
최근 모든 자연어 처리 문제에서 최고의 경지인 SOTA(State od the art)를 기록하고 있는 모델은 BERT(Bidirectional Encoder Representations from Transformers) <br> 

BERT는 "Attention is All You Need" 논문에 등장한 Trnasformer 모델의 Encoder 부분을 기반으로 대량의 코퍼스를 통해 비지도학습으로 범용적인 언어 모델을 학습한 뒤에 풀려고 하는 문제(Task)에 전이 학습(Transfer Learning)을 하는 방법임

## 딥러닝 자연어 처리의 일반적인 프로세스

딥러닝에서 자연어 처리를 수행하기 위해서 텍스트 형식의 데이터를 머신러닝 알고리즘이 이해할 수 있는 숫자<br>
즉, 벡터 형식의 데이터로 만들어 줘야 한다.

1. tokenization - 문장의 토큰화
2. vocabulary - 중복되지 않은 토큰(token)으로 vocabulary 생성 후, 숫자와 1대 1 대응
3. numericalization - 단어장(vocabulary)를 기반으로 토큰(token)을 숫자로 전환 
4. padding - 길이가 다양한 문장을 동일한 길이로 만들어 주는 padding 과정

### 문장이란 tokenization

자연어 처리에서 제일 우선적으로 해야할 작업은 
* 문장(sentence)를 기계가 처리할 수 있는 데이터로 만드는 것

문장(sentence)는 길이가 다른 일련의 토큰(token)의 연속이다.
* 즉, '순서'라는 특징을 가진 시퀀스 데이터(sequence data)라고 할 수 있다.
* 여기서 토큰(token)은 `단어`가 될 수도 있고, `형태소`가 될 수도 있고 `하나의 문자열`이 될 수도 있다.
    * 형태소 : 의미를 가지는 요소로서는 더 이상 분석할 수 없는 가장 작은 말의 단위
    
Example: 문장(sentence) - "오늘 날씨 좋다"<br>

**tokenization**<br>
* token - 단어(띄어쓰기 기준) : '오늘' + '날씨' + '좋다'
* token - 형태소 : '오늘 ' + '날씨' + '좋' + '다'
* token - 문자열 : '오' + '늘' + ' ' + '날' + '씨' + ' ' + '좋' + '다'

**중요한 포인트**<br>
영어의 경우 띄어쓰기로 단어 위주의 토큰화를 할 수 있다.(nltk, SpaCy)(띄어쓰기 기준 split)<br>
한글의 경우 띄어쓰기로 토큰화할 경우 단어 수가 많아지기 때문에 형태소 단위의 토큰화를 기본으로 한다.(KoNLPy)

In [1]:
from konlpy.tag import Okt

In [2]:
kor_sentence = "오늘 날씨 좋다"

In [3]:
tokenizer = Okt()

In [4]:
# morphs 함수로 형태소 분석
tokenizer.morphs(kor_sentence)

['오늘', '날씨', '좋다']

In [5]:
# pos 함수로 품사 태깅
tokenized_sent = tokenizer.pos(kor_sentence)
tokenized_sent

[('오늘', 'Noun'), ('날씨', 'Noun'), ('좋다', 'Adjective')]

In [6]:
# 영어로 된 품사 정보를 한글로 전환
[(k, tokenizer.tagset.get(v)) for k, v in tokenized_sent]

[('오늘', '명사'), ('날씨', '명사'), ('좋다', '형용사')]

### 단어장 생성 vocabulary

단어장(vocabulary)를 생성하는 과정은 중복되지 않는 토큰에 고유한 인덱스(index)를 부여하는 과정이다.
Python에서는 보통 dictionary 자료 구조로 단어장(vocabulary)을 생성한다.
네이버 감성 영화 코퍼스(naver sentiment movie corpus) 데이터를 사용해서 어떻게 진행되는지 알아본다.
이 데이터는 6만개의 댓글과 긍정 혹은 부정을 나타내는 라벨로 구성되어 있으며 그 사이는 tab으로 분리되어 있다.

In [7]:
from konlpy.tag import Okt

In [8]:
tokenizer = Okt()

In [41]:
with open("./data/ratings.txt", encoding='UTF8') as file:
    raw_data = file.read().splitlines()[1:] #첫줄제거
    data = [line.split("\t")[1:] for line in raw_data] #텍스트 데이터와 라벨을 분리
    data = [(tokenizer.morphs(sent), int(label)) for (sent, label) in data] #토큰화 진행

In [37]:
raw_data

['8112052\t어릴때보고 지금다시봐도 재밌어요ㅋㅋ\t1',
 '8132799\t디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.\t1',
 '4655635\t폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.\t1',
 '9251303\t와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지\t1',
 '10067386\t안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.\t1',
 '2190435\t사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화\t1',
 '9279041\t완전 감동입니다 다시봐도 감동\t1',
 '7865729\t개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음\t1',
 '7477618\t굿\t1',
 '9250537\t바보가 아니라 병 쉰 인듯\t1',
 '9730759\t내 나이와 같은 영화를 지금 본 나는 감동적이다..하지만 훗날 다시보면대사하나하나 그 감정을완벽하게 이해할것만 같다...\t1',
 '640794\t재밌다\t1',
 '9537008\t고질라니무 귀엽다능ㅋㅋ\t1',
 '4911311\t영화의 오페라화라고 해야할 작품. 극단적 평갈림은 어쩔 수 없는 듯.\t1',
 '6686673\t3도 반전 좋았제 ^^\t1',
 '9034036\t평점 왜 낮아? 긴장감 스릴감 진짜 최고인데 진짜 전장에서 느끼는 공포를 생생하게 전해준다.\t1',
 '979683\t네고시에이터랑 소재만 같을 뿐.. 아무런 관련없음..\t1',
 '165498\t단연 최고\t1',
 '8703997\t가면 갈수록 더욱 빠져드네요 밀회 화이팅!!\t1',
 '9468781\t어?생각없이 봤는데 상당한 수작.일본영화 10년내 최고로 마음에 들었다.강렬한 임팩트가 일품.\t1',
 '5185638

In [40]:
data

[['어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1'],
 ['디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.',
  '1'],
 ['폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.', '1'],
 ['와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지', '1'],
 ['안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.', '1'],
 ['사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화', '1'],
 ['완전 감동입니다 다시봐도 감동', '1'],
 ['개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음', '1'],
 ['굿', '1'],
 ['바보가 아니라 병 쉰 인듯', '1'],
 ['내 나이와 같은 영화를 지금 본 나는 감동적이다..하지만 훗날 다시보면대사하나하나 그 감정을완벽하게 이해할것만 같다...', '1'],
 ['재밌다', '1'],
 ['고질라니무 귀엽다능ㅋㅋ', '1'],
 ['영화의 오페라화라고 해야할 작품. 극단적 평갈림은 어쩔 수 없는 듯.', '1'],
 ['3도 반전 좋았제 ^^', '1'],
 ['평점 왜 낮아? 긴장감 스릴감 진짜 최고인데 진짜 전장에서 느끼는 공포를 생생하게 전해준다.', '1'],
 ['네고시에이터랑 소재만 같을 뿐.. 아무런 관련없음..', '1'],
 ['단연 최고', '1'],
 ['가면 갈수록 더욱 빠져드네요 밀회 화이팅!!', '1'],
 ['어?생각없이 봤는데 상당한 수작.일본영화 10년내 최고로 마음에 들었다.강렬한 임팩트가 일품.', '1'],
 ['오랜만에 본 제대로 된 범죄스릴러~', '1'],
 ["그런 때가 있었다. ('사랑해'도 아니고) 그저 좋아한다는 한 마디 말을 꺼내기도 벅차서 밤 잠 설치던 때. 커징텅의 교복에 남

In [42]:
data

[(['어릴', '때', '보고', '지금', '다시', '봐도', '재밌어요', 'ㅋㅋ'], 1),
 (['디자인',
   '을',
   '배우는',
   '학생',
   '으로',
   ',',
   '외국',
   '디자이너',
   '와',
   '그',
   '들',
   '이',
   '일군',
   '전통',
   '을',
   '통해',
   '발전',
   '해가는',
   '문화',
   '산업',
   '이',
   '부러웠는데',
   '.',
   '사실',
   '우리나라',
   '에서도',
   '그',
   '어려운',
   '시절',
   '에',
   '끝',
   '까지',
   '열정',
   '을',
   '지킨',
   '노라노',
   '같은',
   '전통',
   '이',
   '있어',
   '저',
   '와',
   '같은',
   '사람',
   '들',
   '이',
   '꿈',
   '을',
   '꾸고',
   '이뤄',
   '나갈',
   '수',
   '있다는',
   '것',
   '에',
   '감사합니다',
   '.'],
  1),
 (['폴리스스토리',
   '시리즈',
   '는',
   '1',
   '부터',
   '뉴',
   '까지',
   '버릴께',
   '하나',
   '도',
   '없음',
   '..',
   '최고',
   '.'],
  1),
 (['와',
   '..',
   '연기',
   '가',
   '진짜',
   '개',
   '쩔구나',
   '..',
   '지루할거라고',
   '생각',
   '했는데',
   '몰입',
   '해서',
   '봤다',
   '..',
   '그래',
   '이런게',
   '진짜',
   '영화',
   '지'],
  1),
 (['안개', '자욱한', '밤하늘', '에', '떠', '있는', '초승달', '같은', '영화', '.'], 1),
 (['사랑', '을', '해본', '사람', '이', '라면', '

In [23]:
sample_data = data[4:14] + data[-10:]

In [33]:
ss = data[0] + data[2]

In [34]:
ss

(['아', '더빙', '..', '진짜', '짜증나네요', '목소리'],
 0,
 ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'],
 0)

분리된 토큰을 사용해 단어장을 생성하는 방법은 중복되지 않은 단어를 제외하고 고유 숫자를 부여하는 방식이다.

데이터에 존재하지 않는 단어도 있기 때문에

단어장에는 '알 수 없음'을 뜻하는 `'<unk>'` 토큰
    
그리고 문장의 길이를 같게 만들어주는 패드 토큰 `'<pad>'`을 단어장에 포함해야함
    

In [ ]:
def build_vocab(data):
    flatten = lambda d: [token for sent in d for token in sent] #list안의 list를 하나의 리스트로 풀어줌
    vocab = {}
    vocab['<unk>'] = 0
    vocab['<pad>'] = 1
    for token in flatten(list(zip(*data))[0]):
        if vocab.get(token) in None:
            vocab.setdefault(token, len(vocab))
    return vocab

vocab = build_vocab()

In [9]:
import MeCab

In [10]:
kor_sentence = "오늘 날씨 좋다"

In [11]:
tokenizer = MeCab.Tagger()

In [12]:
from konlpy.tag import Mecab

In [13]:
mecab = Mecab()

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [ ]:
e